<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [5]</a>'.</span>

In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


##### Import necessary libraries and load environment variables

In [2]:

import os
import snowflake.connector
import boto3
from sqlalchemy import create_engine, MetaData, Table, Column, VARCHAR, TIMESTAMP, INTEGER
from dotenv import load_dotenv
from snowflake.sqlalchemy import URL
from sqlalchemy.types import String, DateTime, Integer

load_dotenv()


False

##### Retrieve Snowflake credentials from environment variables

In [3]:

snowflake_user = os.getenv('SNOWFLAKE_USER')
snowflake_password = os.getenv('SNOWFLAKE_PASSWORD')
snowflake_account = os.getenv('SNOWFLAKE_ACCOUNT')
warehouse = 'SF_WH_CASE1'
database = 'SF_DB_CASE1'
schema = 'SF_CASE1'
role = 'SYSADMIN'  # Replace with your full access role


##### Retrieve S3 credentials from environment variables

In [4]:

aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')


##### Connect to Snowflake and create a cursor object

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [5]:

conn = snowflake.connector.connect(
    user=snowflake_user,
    password=snowflake_password,
    account=snowflake_account,
    warehouse=warehouse,
    database=database,
    schema=schema,
    role=role,
)
cur = conn.cursor()


ProgrammingError: 251005: User is empty

##### Execute SQL commands for setup

In [ ]:

try:
    cur.execute("CREATE DATABASE IF NOT EXISTS SF_DB_CASE1")
    cur.execute("USE DATABASE SF_DB_CASE1")
    cur.execute("""
        CREATE WAREHOUSE IF NOT EXISTS SF_WH_CASE1
        WITH WAREHOUSE_SIZE = 'MEDIUM'
        WAREHOUSE_TYPE = 'STANDARD'
        AUTO_SUSPEND = 300
        AUTO_RESUME = TRUE;
    """)
    cur.execute("CREATE SCHEMA IF NOT EXISTS SF_CASE1")
    cur.execute("USE SCHEMA SF_CASE1")
    cur.execute("""
        CREATE OR REPLACE TABLE METADATA (
            OBJECT_NAME VARCHAR(255),
            LAST_MODIFIED TIMESTAMP,
            SIZE_BYTES NUMBER,
            LINK_TO_TXT_FILE VARCHAR(1000)
        );
    """)
    print("Snowflake setup completed successfully.")
except Exception as e:
    print(e)
finally:
    # Always close the cursor
    cur.close()


##### Initialize S3 client

In [ ]:

s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)


##### Specify S3 bucket and prefix and list objects

In [ ]:

bucket_name = 'bigdata-pypdf'
prefix = 'PyPDF/'
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)


In [ ]:
engine = create_engine(URL(
    account = snowflake_account,
    user = snowflake_user,
    password = snowflake_password,
    database = database,
    schema = schema,
    warehouse = warehouse,
    role = role,
))

metadata = MetaData()


metadata_table = Table(
    'METADATA', metadata,
    Column('OBJECT_NAME', String(255)),
    Column('LAST_MODIFIED', DateTime),
    Column('SIZE_BYTES', Integer),
    Column('LINK_TO_TXT_FILE', String(1000))
)

metadata.create_all(engine, checkfirst=True)

##### Metadata transfer to Snowflake

In [ ]:

try:
    with engine.connect() as connection:
        for obj in response.get('Contents', []):
            object_name = obj['Key']
            last_modified = obj['LastModified']
            size_bytes = obj['Size']
            link_to_txt_file = f"https://{bucket_name}.s3.amazonaws.com/{object_name}"
            connection.execute(metadata_table.insert().values(
                OBJECT_NAME=object_name,
                LAST_MODIFIED=last_modified,
                SIZE_BYTES=size_bytes,
                LINK_TO_TXT_FILE=link_to_txt_file
            ))
    print("Metadata transfer to Snowflake completed successfully.")
except Exception as e:
    print(e)


In [ ]:

conn.close()
